
# Basic Overview
The objective is to build relevant random forest models and predict the survival rates for people on board the RMS Titanic !

Comments/criticisms/appreciations are greatly accepted and appreciated. Do not be shy and send me an email at babinu@gmail.com !

Source of data : https://www.kaggle.com/c/titanic/data

In [144]:
import pandas as pd
import numpy as np

In [145]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [146]:
# Add a column called Predictions in training data, which will become handy while doing
# cross validation.
train_data['Predictions'] = -1

In [147]:
# Using h2o
import h2o
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 56 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,4 months and 4 days !!!
H2O cluster name:,H2O_from_python_babs4JESUS_p6ymx9
H2O cluster total nodes:,1
H2O cluster free memory:,6.774 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [148]:
def get_h2o_frame_with_rel_factors_test(clean_training_data_v3):
    clean_h2o_data = h2o.H2OFrame(clean_training_data_v3)    
    if 'Sex' in clean_training_data_v3.columns:
        clean_h2o_data['Sex'] = clean_h2o_data['Sex'].asfactor()
    if 'Pclass' in clean_training_data_v3.columns:
        clean_h2o_data['Pclass'] = clean_h2o_data['Pclass'].asfactor()
    if 'IsAlone' in clean_training_data_v3.columns:
        clean_h2o_data['IsAlone'] = clean_h2o_data['IsAlone'].asfactor()    
    return clean_h2o_data

In [149]:
def get_h2o_frame_with_rel_factors(clean_training_data_v3):
    
    clean_h2o_data = get_h2o_frame_with_rel_factors_test(clean_training_data_v3)
    if 'Survived' in clean_training_data_v3.columns:
        clean_h2o_data['Survived'] = clean_h2o_data['Survived'].asfactor()
    return clean_h2o_data

In [150]:
train_data_h2o = get_h2o_frame_with_rel_factors(train_data)
test_data_h2o = get_h2o_frame_with_rel_factors_test(test_data)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [151]:
# Function for splitting input dataframe into training/validation and test sets.
def get_train_validation_test_data(input_df, ratio=0.75):
    train_len = int(ratio * len(input_df))
    train_validation = input_df[:train_len] 
    test = input_df[train_len:]
    return (train_validation, test)


In [152]:
def get_predictions_given_model_and_data(rf_fit, train_validation, train_validation_h2o):
    # Using relevant routine from scikit learn.
    predict_out = rf_fit.predict(train_validation_h2o) 
    if 'Survived' in train_validation.columns:
        survived_set = pd.DataFrame(columns=['Survived', 'Predictions'])
        survived_set['Survived'] = train_validation['Survived']
    else:
        survived_set = pd.DataFrame(columns=['Predictions'])
    survived_set['Predictions'] = predict_out.as_data_frame()['predict'].values.tolist()
    
    return survived_set

In [153]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator
from sklearn.metrics import roc_curve, auc, roc_auc_score


In [154]:
from sklearn.preprocessing import Imputer
# The bread and butter routine for cross validation.
def get_cross_val_output_mean_imputation(input_df, x_columns=['Sex', 'Pclass', 'Age'], 
                                          y_column = 'Survived', nfolds=10):
    partition_indices = np.array_split(np.arange(len(input_df)), nfolds)
    cross_validated_data = pd.DataFrame(columns=input_df.columns)
    for i in range(nfolds):
        cross_validated_set = input_df[partition_indices[i][0]:partition_indices[i][-1] + 1].copy()
        rel_training_data = pd.DataFrame(columns=input_df.columns)
        for j in range(nfolds):
            if j != i:
                training_set = input_df[partition_indices[j][0]:partition_indices[j][-1] + 1]
                rel_training_data = pd.concat([rel_training_data, training_set])

        # Make sure that a copy is made, so that the original dataset is untouched.
        rel_training_data_v1 = rel_training_data.copy()

        mean_age_training_data = rel_training_data_v1['Age'].mean()
        # Impute with the mean and get a new training set(and a test set)
        rel_training_data_v1.loc[
            (rel_training_data_v1['Age'].isnull()), 'Age'] = mean_age_training_data
        my_imputer = Imputer()
        rel_training_data_v1['Age'] = my_imputer.fit_transform(rel_training_data_v1[['Age']])
        cross_validated_set['Age'] = my_imputer.transform(cross_validated_set[['Age']])

        h2o_train_data = get_h2o_frame_with_rel_factors(rel_training_data_v1)

        rf_fit = H2ORandomForestEstimator(model_id='rf_fit', seed=1)
        rf_fit.train(x=x_columns, y=y_column, training_frame=h2o_train_data)

        predict_out = rf_fit.predict(
            get_h2o_frame_with_rel_factors_test(cross_validated_set))                

        cross_validated_set['Predictions'] = predict_out.as_data_frame()['predict'].values.tolist()
        
        cross_validated_data = pd.concat([cross_validated_data, cross_validated_set])


    return cross_validated_data
    

In [155]:
def display_manual_model_metrics(test_v1, message):
    print("Fit score obtained  :" + message, 
          np.sum((test_v1['Survived'] == test_v1['Predictions'])/len(
              test_v1['Survived'])))
    print("Number of correct entries is {0}. Total number of entries is {1}".format(
        np.sum(test_v1['Survived'] == test_v1['Predictions']), len(test_v1['Survived'])))

    print("AUC obtained  :" + message, 
          roc_auc_score(test_v1['Survived'].apply(lambda x : int(x)),
                        test_v1['Predictions'].apply(lambda x : int(x))))    

In [156]:
def get_normalized_training_test_data(train_validation, test):
    train_validation_v1 = train_validation.copy()
    test_v1 = test.copy()

    mean_age_training_data = train_validation_v1['Age'].mean()
    # Impute with the mean and get a new training set(and a test set)
    train_validation_v1.loc[
        (train_validation_v1['Age'].isnull()), 'Age'] = mean_age_training_data
    test_v1.loc[(test_v1['Age'].isnull()), 'Age'] = mean_age_training_data
    train_validation_v1_h2o = get_h2o_frame_with_rel_factors(train_validation_v1)
    test_v1_h2o = get_h2o_frame_with_rel_factors_test(test_v1)
    return (train_validation_v1, test_v1, train_validation_v1_h2o, test_v1_h2o)


In [157]:
def get_trained_model_with_given_data(train_validation_v1_h2o, 
                                      x_columns = ['Sex', 'Pclass', 'Age'], 
                                      y_column = 'Survived'):
    model4_full_train_data = H2ORandomForestEstimator(model_id='model4_full_train_data', seed=1)
    model4_full_train_data.train(x=x_columns, y=y_column, 
                                 training_frame=train_validation_v1_h2o)
    return model4_full_train_data

In [158]:
def normalize_training_data_and_build_model(train_validation, test, x_columns=['Sex', 'Pclass', 'Age'], y_column='Survived'):
    (train_validation_v1, test_v1, train_validation_v1_h2o, test_v1_h2o) = \
        get_normalized_training_test_data(train_validation, test)
    model4_full_train_data = get_trained_model_with_given_data(train_validation_v1_h2o, x_columns, y_column)
    predict_out = get_predictions_given_model_and_data(model4_full_train_data, test_v1, test_v1_h2o)
    return (test_v1, predict_out)

In [159]:
# Training the model on entire data. 
def train_validate_and_test_model(train_validation, test, x_columns=['Sex', 'Pclass', 'Age'], y_column='Survived'):
    (test_v1, predict_out) = normalize_training_data_and_build_model(train_validation, 
                                                                     test, 
                                                                     x_columns, 
                                                                     y_column)
    test_v1['Predictions'] = predict_out['Predictions']
    display_manual_model_metrics(test_v1, " (while making predictions on the test set) ")

In [160]:
# Training the model on entire data. 
def train_full_data_and_make_predictions_out_of_sample(train_data, 
                                                       test_data, 
                                                       x_columns=['Sex', 'Pclass', 'Age'], 
                                                       y_column='Survived'):
    (test_v1, predict_out) = normalize_training_data_and_build_model(train_data, 
                                                                     test_data, 
                                                                     x_columns, 
                                                                     y_column)
    test_v1['Survived'] = predict_out['Predictions']
    return test_v1


In [161]:
def cross_validate_train_test_data(train_data, 
                                   ratio=0.75,
                                   x_columns=['Sex', 'Pclass', 'Age'], 
                                   y_column='Survived', 
                                   nfolds=10):
    (train_validation, test) = get_train_validation_test_data(train_data)

    cross_validated_output = get_cross_val_output_mean_imputation(train_validation, 
                                                                  x_columns,
                                                                  y_column,
                                                                  nfolds)
    display_manual_model_metrics(cross_validated_output, " (during cross validation) ")
    test_v1 = train_validate_and_test_model(train_validation, 
                                            test, 
                                            x_columns, 
                                            y_column)
    return test_v1

### Model 1 : Use Sex alone

In [162]:
cross_validate_train_test_data(train_data, 0.75, ['Sex'], 'Survived') 

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

### Model 2 : Add Pclass as another factor.

In [163]:
cross_validate_train_test_data(train_data, 0.75, ['Sex', 'Age'], 'Survived') 

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

### Model 3 : Add Age as another factor.


In [164]:
cross_validate_train_test_data(train_data, 0.75, ['Sex', 'Age', 'Pclass'], 'Survived') 

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

### Model 3 : Add Parch as another factor.


In [165]:
cross_validate_train_test_data(train_data, 0.75, ['Sex', 'Pclass', 'Age', 'Parch'], 'Survived')

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

We look to be getting some benefit with respect to the addition of Parch factor. Let us generate our predictions to kaggle !

In [166]:
test_data_with_predictions = train_full_data_and_make_predictions_out_of_sample(train_data, 
                                                                                test_data, 
                                                                                x_columns=['Sex', 'Pclass', 'Age', 'Parch'], 
                                                                                y_column='Survived')
test_data_with_predictions[['PassengerId', 'Survived']].sort_values(
    by=['PassengerId']).to_csv("output_sex_pclass_age_parch1.csv", index=False)

# Kaggle score : 0.73205 :(

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


### Model 4 : Add SibSp as another factor.

Let us see  if adding sibling related information would help us.

In [167]:
cross_validate_train_test_data(train_data, 0.75, ['Sex', 'Pclass', 'Age', 'Parch', 'SibSp'], 'Survived')

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

### Model 5 : Add number of family members as another factor.

Now, let us try adding the number of members of the family and see where we get.

In [168]:
# Add another factor on familyNum
train_data['FamilyNum'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['FamilyNum'] = test_data['SibSp'] + test_data['Parch'] + 1



In [169]:
cross_validate_train_test_data(train_data, 
                               0.75, 
                               ['Sex', 'Pclass', 'Age', 'FamilyNum'], 
                               'Survived')

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

### Model 6 : Add the information whether the passenger was traveling alone as another indicator.

Check whether the fact whether the passenger was traveling alone or not would increase predictive performance.

In [170]:
train_data['IsAlone'] = (train_data['FamilyNum'] == 1)
test_data['IsAlone'] = (test_data['FamilyNum'] == 1)

In [171]:
cross_validate_train_test_data(train_data, 
                               0.75, 
                               ['Sex', 'Pclass', 'Age', 'IsAlone'], 
                               'Survived')

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

### Model 7 : Add  farePerPerson information.
Since, we have fare as well as the number of members in the family, we can construct a fare per person variable and see how that helps us.

In [172]:
# Check for null values initially.
train_data[train_data['Fare'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Predictions,FamilyNum,IsAlone


In [173]:
train_data[train_data['FamilyNum'] == 0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Predictions,FamilyNum,IsAlone


In [174]:
train_data['FarePerPerson'] = train_data['Fare']/train_data['FamilyNum']

In [175]:
cross_validate_train_test_data(train_data, 
                               0.75, 
                               ['Sex', 'Pclass', 'Age', 'FarePerPerson'], 
                               'Survived')

Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |███████████

### Conclusion.

By cross validation results, it looks like the performance of the model with factors Sex, Pclass, Age and Parch was the best. Hence, we proceed on to generate predictions on out of sample data using the same.

In [176]:
test_data_with_predictions = \
    train_full_data_and_make_predictions_out_of_sample(train_data, 
                                                       test_data, 
                                                       x_columns=['Sex', 'Pclass', 'Age', 'Parch'], 
                                                       y_column='Survived')
test_data_with_predictions[['PassengerId', 'Survived']].sort_values(
    by=['PassengerId']).to_csv("output_sex_pclass_age_parch1.csv", index=False)

# Kaggle score : 0.73205 :(

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
